<a href="https://colab.research.google.com/github/christopherdiamana/nlp/blob/main/catch-up1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Natural Language Processing Catch-up 1

## The dataset

In [1]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 5.3 MB/s 
     |████████████████████████████████| 101 kB 6.8 MB/s 
     |████████████████████████████████| 140 kB 46.4 MB/s 
     |████████████████████████████████| 212 kB 16.3 MB/s 
     |████████████████████████████████| 1.1 MB 38.9 MB/s 
     |████████████████████████████████| 596 kB 44.9 MB/s 
     |████████████████████████████████| 127 kB 46.7 MB/s 
     |████████████████████████████████| 144 kB 50.4 MB/s 
     |████████████████████████████████| 271 kB 57.4 MB/s 
     |████████████████████████████████| 94 kB 3.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's depe

In [2]:
from datasets import load_dataset_builder

In [3]:
ds_builder = load_dataset_builder("imdb")

In [4]:
# Inspect dataset description
ds_builder.info.description

'Large Movie Review Dataset.\nThis is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.'

In [5]:
# Inspect dataset features
ds_builder.info.features

{'label': ClassLabel(num_classes=2, names=['neg', 'pos'], id=None),
 'text': Value(dtype='string', id=None)}

In [6]:
from datasets import get_dataset_split_names

In [7]:
get_dataset_split_names("imdb")

['train', 'test', 'unsupervised']

In [8]:
from datasets import load_dataset

In [9]:
dataset = load_dataset("imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [11]:
imdb_train = load_dataset('imdb', split='train')
imdb_test = load_dataset('imdb', split='test')

Reusing dataset imdb (/root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)
Reusing dataset imdb (/root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


In [12]:
from collections import Counter

In [13]:
Counter(imdb_train['label'])

Counter({0: 12500, 1: 12500})

In [14]:
Counter(imdb_test['label'])

Counter({0: 12500, 1: 12500})



---



## Naive Bayes classifier

In [15]:
import math
from operator import countOf


def train_naive_bayes(documents, classes):
  log_prior = {}
  vocabulary = []
  likelihood = {{}}

  for class_name in classes:
    num_documents = len(documents['text'])
    num_documents_of_class = len([document_class for document_class in documents['label'] if document_class == class_name])

    log_prior[class_name] = math.log(num_documents_of_class / num_documents)

    
    [vocabulary.append(document.split()) for document in documents]

    big_document = []
    [big_document.append(document['text']) for document in documents if document['label'] == class_name]

    for current_word in vocabulary:
      count = countOf(big_document, current_word)
      
      numerator = count + 1
      denominator = sum([countOf(big_document, word)  for word in vocabulary])+ 1
      likelihood[current_word][class_name] = math.log(numerator / denominator)

  return log_prior, likelihood, vocabulary

In [16]:
import numpy as np


def test_naive_bayes(test_documents, log_prior, loglikelihood, classes, vocabulary):
  summation = []

  for class_name in classes:
    summation[class_name] = log_prior[class_name].value
    for word in test_documents:
      #word = test_documents[i]
      if word in vocabulary:
        summation[class_name] += loglikelihood[word][class_name] 
  
  return np.argmax(summation)

### Pretreatment

In [ ]:
from string import punctuation
import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [ ]:
def pre_treatment(text):
  new_text = text.lower()
  new_text = new_text.translate(str.maketrans('', '', punctuation))
  
  #Other pretreatments
  new_text = re.sub(r'[^a-z]+', ' ', new_text)
  new_text = re.sub(r'\b\w\b', ' ', new_text)
  new_text = re.sub(r'\b\w\w\b', ' ', new_text)
  word_tokens = word_tokenize(new_text)
  lemmatizer = WordNetLemmatizer()

  return ' '.join([lemmatizer.lemmatize(w) for w in word_tokens])

In [ ]:
from gensim.parsing.preprocessing import STOPWORDS

#Creating a list of custom stopwords
new_words = ["fig","figure","image","sample","using", 
             "show", "result", "large", 
             "also", "one", "two", "three", 
             "four", "five", "seven","eight","nine"]

stop_words = STOPWORDS.union(set(new_words))

In [ ]:
{idx:label for idx, label in enumerate(labels)}

In [ ]:
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}